#  Setup: Display options

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


# Setup: Connectivity

In [ ]:

#  Settings:
#     Need a public or private accessible storage bucket on GCP or similar ..
#
#  From,
#     https://stackoverflow.com/questions/36314797/write-a-pandas-dataframe-to-google-cloud-storage-or-bigquery
#     https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29325954#29325954

import os
from google.cloud import storage
import pandas as pd

print ("--")

In [ ]:

#  Setup for all work below ..

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jovyan/work/03_MyKeys.json"

l_bucket = "farrell-bucket"


g_client = storage.Client()
   #
g_bucket = g_client.get_bucket(l_bucket)

print("--")


In [ ]:
# ##################################################################

In [ ]:
import os

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS = 3
   #
DB_NAME        = "my_db"
GRAPH_NAME     = "my_graph"

print("--")


In [ ]:
# ##################################################################

In [ ]:
#  CREATE DATABASE
#
#  repeat execution throws error

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)

In [ ]:
#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=2)

print(my_graph)

In [ ]:
#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:
# ##################################################################

In [ ]:

display(my_graph.num_nodes())
display(my_graph.num_edges())


In [ ]:
# ##################################################################

In [ ]:

#  We need a header/schema file to import into a graph ..

df_h = pd.DataFrame([
   ["KATANA_DEFAULT_LABEL=Node1"                       ],
   ["id:ID(Node1)|name:STRING|other:STRING|city:STRING"],
   ["load_test/node.parquet"                          ],
   ], columns = ["Col_A"])

l_SchemaFile = "load_test/test_header.P.txt"

g_bucket.blob(l_SchemaFile).upload_from_string(df_h.to_csv(header=None, index=None, sep=" "), "text/plain")


df_h = pd.DataFrame([
   ["KATANA_DEFAULT_LABEL=Node1"                       ],
   ["id:ID(Node1)|name:STRING|other:STRING|city:STRING"],
   ["load_test/node.txt"                               ],
   ], columns = ["Col_A"])

l_SchemaFile = "load_test/test_header.C.txt"

g_bucket.blob(l_SchemaFile).upload_from_string(df_h.to_csv(header=None, index=None, sep=" "), "text/plain")


print("--")

In [ ]:

#  Simple data ..

#  Set (i = 0) for a given data set, set to (i = 1) for a second data set
#  (Used to test updates, after initial load, other)

i = 0
# i = 1

if (i == 0):
   df = pd.DataFrame([
      [101, "Abhi"      , "Katana"                , 'NY'     ],
      [102, "Justin"    , "Katana"                , 'LA'     ],
      [103, "Brian"     , "Katana"                , 'Denver' ],
      [104, "Thomas"    , "Katana"                , 'Denver' ],
      [105, "Anand"     , "Katana"                , 'Denver' ],
      [106, "Josh"      , "Katana"                , 'Denver' ],
      [107, "Andrew"    , "Katana"                , 'Denver' ],
      [108, "Seb"       , "Katana"                , 'Denver' ],
      [109, "Mary"      , "Katana"                , 'Denver' ],
      [100, "Pahola"    , "Katana"                , 'Denver' ],
      ], columns = ['Col A', 'Col B', 'Col C', 'Col D'])
else:
   df = pd.DataFrame([
      [201, "Abhi 2"    , "Katana"                , 'NY'     ],
      [202, "Justin 2"  , "Katana"                , 'LA'     ],
      [203, "Brian 2"   , "Katana"                , 'Denver' ],
      [204, "Thomas 2"  , "Katana"                , 'Denver' ],
      [205, "Anand 2"   , "Katana"                , 'Denver' ],
      [206, "Josh 2"    , "Katana"                , 'Denver' ],
      [207, "Andrew 2"  , "Katana"                , 'Denver' ],
      [208, "Seb 2"     , "Katana"                , 'Denver' ],
      [209, "Mary 2"    , "Katana"                , 'Denver' ],
      [200, "Pahola 2"  , "Katana"                , 'Denver' ],
      ], columns = ['Col A', 'Col B', 'Col C', 'Col D'])  


l_file = "load_test/node.parquet"
   #
g_bucket.blob(l_file).upload_from_string(df.to_parquet(engine="pyarrow", version="2.6"), "application/octet-stream")

l_file = "load_test/node.txt"
   #
g_bucket.blob(l_file).upload_from_string(df.to_csv(header=None, index=None, sep="|"), "text/plain")


print("--")

In [ ]:

#  Simple test, read output from above ..

#  Set (i = P) for Parquet, (i = C) for CSV

# i = "P"
i = "C"

if (i == "P"):
   l_file = "load_test/node.parquet"
      #
   l_InputFile  = "gs://" + l_bucket + "/" + l_file
      #
   l_df = pd.read_parquet(l_InputFile)
else:
   l_file = "load_test/node.txt"
      #
   l_InputFile  = "gs://" + l_bucket + "/" + l_file
      #
   l_df = pd.read_csv(l_InputFile, delimiter = "|", header = None)


l_df.head(5)


In [ ]:

#  Below is the key cell ..
#
#   .  Upsert when new graph does work
#      Repeat the upsert, (so now data is present) and it fails
#
#   .  Basically any new data only situation below works
#      Any work on existing data will fail
#
#      Update on very same data (so no actual updating), works
#
#   .  And strangely, delete works on existing data



In [ ]:

#  Make a graph from all above ..


#  Set (i = P) to load Parquet into graph, (i = C) to load from CSV 

# i = "P"
i = "C"


l_data = "gs://" + l_bucket 


if (i == "P"):
   l_inp  = "gs://" + l_bucket + "/load_test/test_header.P.txt"
      #
   import_data.parquet(
      my_graph,
      input_node_path=l_inp,
      input_dir=l_data,
      schema_delimiter="|",
      ids_are_integers=True,
      operation=import_data.Operation.Insert,
      # operation=import_data.Operation.Delete,
      # operation=import_data.Operation.Update,
      # operation=import_data.Operation.Upsert,
      )
else:
   l_inp  = "gs://" + l_bucket + "/load_test/test_header.C.txt"
      #
   import_data.csv(
      my_graph,
      input_node_path=l_inp,
      input_dir=l_data,
      schema_delimiter="|",
      data_delimiter = "|",
      ids_are_integers=True,
      files_have_headers = False,
         #
      # operation=import_data.Operation.Insert,
      # operation=import_data.Operation.Delete,
      # operation=import_data.Operation.Update,
      operation=import_data.Operation.Upsert,
      )

print("--")


In [ ]:
my_graph.num_nodes()

In [ ]:
assert my_graph.num_nodes() == 10

In [ ]:

l_result = my_graph.query("""

   MATCH (n)
   RETURN n
   
   """)

print(l_result)
